# Memoirr: Preprocessor + Chunker Pipeline Smoke Test

This notebook runs a minimal Haystack pipeline using the SRT preprocessor and the semantic chunker.

Requirements:
- Ensure a local sentence-transformers model is available under `models/<EMBEDDING_MODEL_NAME>/` (with `model.safetensors` and tokenizer/config files).
- `.env` should set `EMBEDDING_MODEL_NAME` (default included in repo). Optionally set `EMBEDDING_DEVICE` (e.g., `cuda:0`).

Notes:
- The preprocessor emits cleaned JSONL lines, one per caption.
- The chunker uses Chonkie SemanticChunker with the self-hosted embeddings to create time-aware chunks.


In [ ]:

import pathlib
import textwrap
from src.core.config import get_settings

settings = get_settings()
print('EMBEDDING_MODEL_NAME =', settings.embedding_model_name)
print('EMBEDDING_DEVICE     =', settings.device)

# Quick existence check to help the user
model_path = pathlib.Path('models') / settings.embedding_model_name
if not model_path.exists():
    # Fallback: search by terminal folder name (case-insensitive), similar to runtime resolver
    target = settings.embedding_model_name.split('/')[-1].lower()
    candidates = [p for p in pathlib.Path('models').rglob('*') if p.is_dir() and p.name.lower() == target]
    if candidates:
        print('Found candidate model dir at:', candidates[0])
    else:
        print('WARNING: Expected model folder not found under models/. The chunker cell may fail.')


In [ ]:
# Sample SRT content (very small)
sample_srt = textwrap.dedent('''
1
00:00:01,000 --> 00:00:02,000
- Hello there!

2
00:00:02,100 --> 00:00:03,000
How are you doing?

3
00:00:03,100 --> 00:00:04,000
I'm fine. Thanks!
''')
print(sample_srt)


In [ ]:
# Run the complete end-to-end pipeline: SRT → Preprocess → Chunk → Embed → Qdrant
from src.pipelines.srt_to_qdrant import build_srt_to_qdrant_pipeline

print('Building the complete SRT-to-Qdrant pipeline...')
pipe = build_srt_to_qdrant_pipeline()

print('Pipeline components:')
for component_name in pipe.graph.nodes:
    print(f'  - {component_name}')

print('Pipeline connections:')
for edge in pipe.graph.edges:
    print(f'  - {edge[0]} → {edge[1]}')

print('Running pipeline on sample SRT...')
result = pipe.run({'pre': {'srt_text': sample_srt}})

print('Pipeline Results:')
print('=================')

# Show preprocessing stats
pre_stats = result['pre']['stats']
print(f'Preprocessor: {pre_stats}')

# Show chunking stats
chunk_stats = result['chunk']['stats']
print(f'Chunker: {chunk_stats}')

# Show write stats
write_stats = result['write']['stats']
print(f'Writer: {write_stats}')

print('✅ SUCCESS! Check Qdrant UI at http://localhost:6300/dashboard to see the embedded chunks!')
print('Collection name: memoirr')

In [ ]:
# Retrieval Pipeline Test
# This script demonstrates how to use the RAG pipeline to retrieve and generate answers from
# indexed content

from src.pipelines.rag_pipeline import RAGPipeline

# Initialize the RAG pipeline
print("Initializing RAG pipeline...")
rag = RAGPipeline()

# Test basic retrieval with a sample query
print("\nTesting basic retrieval...")
query = "What is the main character's name?"

try:
    result = rag.query(query)

    print(f"Query: {query}")
    print(f"Retrieved documents: {result['summary']['documents_retrieved']}")
    print(f"Generated replies: {result['summary']['replies_generated']}")

    if result.get('answer'):
        print(f"Answer: {result['answer']}")

    # Show source documents
    if result.get('sources'):
        print("\nTop source documents:")
        for i, source in enumerate(result['sources'][:3]):  # Show top 3 sources
            print(f"  {i+1}. {source['content'][:100]}... (score: {source['score']:.3f})")

except Exception as e:
    print(f"Error during retrieval: {e}")

print("\n✅ Retrieval pipeline tests completed!")


In [1]:
import torch
torch.cuda.is_available()

/home/maarten/Documents/data-science/memoirr/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:182: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


False

In [2]:
from src.database_population import process_srt_directory

2025-10-03 21:53:54 [info     ] PyTorch CUDA memory configuration set component=memory_management config=expandable_segments:True,garbage_collection_threshold:0.8,max_split_size_mb:128
{"module": "pipelines", "available_pipelines": ["srt_to_qdrant"], "event": "Pipelines module initialized", "level": "info", "timestamp": "2025-10-03T19:53:54.727210Z", "service": "memoirr", "environment": "development"}


In [3]:
process_srt_directory('data', overwrite=True)

2025-10-03T19:54:02.138024Z [info     ] batch_srt_processing_started   directory=data operation=batch_srt_processing overwrite=True


{'operation': 'batch_srt_processing', 'directory': 'data', 'overwrite': True, 'event': 'batch_srt_processing_started', 'level': 'info', 'timestamp': '2025-10-03T19:54:02.138024Z'}


2025-10-03T19:54:02.139268Z [info     ] Starting batch SRT processing  component=batch_processor directory=data overwrite=True


{'directory': 'data', 'overwrite': True, 'component': 'batch_processor', 'event': 'Starting batch SRT processing', 'level': 'info', 'timestamp': '2025-10-03T19:54:02.139268Z'}


2025-10-03T19:54:02.140741Z [info     ] SRT file discovery completed   component=batch_processor directory=data files_found=3


...

2025-10-03T19:56:10.745662Z [info     ] Batch SRT processing completed component=batch_processor directory=data failed_files=0 processing_time_ms=128607 successful_files=3 total_documents_written=1233 total_files=3


{'directory': 'data', 'total_files': 3, 'successful_files': 3, 'failed_files': 0, 'total_documents_written': 1233, 'processing_time_ms': 128607, 'component': 'batch_processor', 'event': 'Batch SRT processing completed', 'level': 'info', 'timestamp': '2025-10-03T19:56:10.745662Z'}


2025-10-03T19:56:10.746897Z [info     ] batch_srt_processing_completed directory=data duration_ms=128608 failed_files=0 operation=batch_srt_processing overwrite=True success_rate=1.0 successful_files=3 total_documents_written=1233 total_files=3


{'operation': 'batch_srt_processing', 'duration_ms': 128608, 'directory': 'data', 'overwrite': True, 'total_files': 3, 'successful_files': 3, 'failed_files': 0, 'total_documents_written': 1233, 'success_rate': 1.0, 'event': 'batch_srt_processing_completed', 'level': 'info', 'timestamp': '2025-10-03T19:56:10.746897Z'}


BatchProcessingResult(total_files=3, successful_files=3, failed_files=0, total_documents_written=1233, file_results=[ProcessingResult(file_path='data/the.lord.of.the.rings.the.fellowship.of.the.ring.(2001).eng.1cd.(6053820)/The.Lord.of.the.Rings.The.Fellowship.of.the.Ring.Extended.Editions.2001.1080p.BluRay.x264.srt', success=True, documents_written=420, error_message=None, processing_time_ms=42141), ProcessingResult(file_path='data/the.lord.of.the.rings.the.return.of.the.king.(2003).eng.1cd.(6256239)/The.Lord.of.the.Rings.The.Return.of.the.King.EXTENDED.2003.720p.BrRip.x264.BOKUTOX.YIFY.Perusoe.srt', success=True, documents_written=367, error_message=None, processing_time_ms=38327), ProcessingResult(file_path='data/the.lord.of.the.rings.the.two.towers.(2002).eng.1cd.(5581877)/The.Lord.of.the.Rings.The.Two.Towers.Extended.Editions.2002.1080p.BluRay.x264.DTS-WiKi .srt', success=True, documents_written=446, error_message=None, processing_time_ms=43354)], processing_time_ms=128607)